In [32]:
import psycopg2
from db_functions import connect_to_db
from cleaning_functions import clean_tweet,tokenize_text

#POST TABLE ATTRIBUTES
POST_TABLE="post"
CANDIDATE_TABLE="candidate"
CANDIDATE_ID="cand_id"
COLLECTION_DATE="collection_date"
CANDIDATE_NAME="c_name"
TEXT="p_text"
EMOTION="em_rate"


#GET ALL AVALILABLE DATES
def get_tweets(*args,**kwargs):
    conn=connect_to_db()
    
    #SET TWEETS TIME INTERVAL
    if len(kwargs)==0:
        #IN CASE OF NOT SPECIFYNG A TIME INTERVAL, ALL TWEETS ARE GOTTEN
        with conn.cursor() as cur:
            cur.execute("SELECT * FROM last_date")
            last_date=cur.fetchone()[0]
            cur.execute("SELECT * FROM first_date")
            first_date=cur.fetchone()[0]
            kwargs["FROM_DATE"]=first_date.strftime('%Y-%m-%d')
            kwargs["TO_DATE"]=last_date.strftime('%Y-%m-%d')
    else:
        with conn.cursor() as cur:
            kwargs["FROM_DATE"]=kwargs["FROM_DATE"]
            kwargs["TO_DATE"]=kwargs["TO_DATE"]



    if len(args)==1: #ADJUST TUPLE TO STRING DEPENDING OF ITS LENGHT
        string_tuple=f'''('{args[0]}')'''
        #print(string_tuple)
    else:
        string_tuple=str(args)
        #print(string_tuple)
   
   #QUERY
    with conn.cursor() as cur:
        query=f'''SELECT p.{TEXT},p.{COLLECTION_DATE},p.{EMOTION} FROM {POST_TABLE} AS p
        JOIN {CANDIDATE_TABLE} AS c USING({CANDIDATE_ID})
        WHERE (c.{CANDIDATE_NAME} IN {string_tuple}) AND (p.{COLLECTION_DATE} BETWEEN '{kwargs["FROM_DATE"]}' AND '{kwargs["TO_DATE"]}')
        ''' 
        cur.execute(query)
        tweets=cur.fetchall()
    conn.close()
    print(tweets)
    return tweets #RETURNS A LIST OF TUPLES


def get_meaningful_words(list_of_tuples):
    
    relevant_words=[]
    for tweet in list_of_tuples:
        #CLEAN TWEETS
        cleaned_tweet=clean_tweet(tweet[0])
        #print(cleaned_tweet)

        #TOKENIZE
        tokenized_text=tokenize_text(cleaned_tweet)
        #print(tokenized_text)

        #GET RELEVANT WORDS
        relevant_words.extend(tokenized_text)
        
    return relevant_words #RETURNS A LIST OF MEANINGFUL WORDS



#ltup=get_tweets("MIGUEL TORRUCO GARZA",FROM_DATE="2023-12-01",TO_DATE="2024-01-07")




The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.


In [37]:
class classifier:
    
    def __init__(self,person_name,tweets):
        self.tweets=tweets
        self.person_name=person_name
        self.positive_tweets={}
        self.negative_tweets={}
        self.neutral_tweets={}
        self.none_tweets={}
        self.dates_dict=[]

    def set_dates_dict(self):
        self.dates_dict={}
        self.dates_dict["POSITIVO"]=list(self.positive_tweets.keys())
        self.dates_dict["NEGATIVO"]=list(self.negative_tweets.keys())
        self.dates_dict["NEUTRO"]=list(self.neutral_tweets.keys())
        self.dates_dict["NULO"]=list(self.none_tweets.keys())
            
        
    def set_none_tweets(self):
        for text,date,emotion in self.tweets:
            date=date.strftime('%Y-%m-%d')
            if emotion is None and date not in self.none_tweets:
                    self.none_tweets[date]=[text]
                    #print(self.none_tweets)
            elif emotion is None and date in self.none_tweets:
                    self.none_tweets[date].append(text)
                    #print(self.none_tweets)
                    
    def set_positive_tweets(self):
        for text,date,emotion in self.tweets:
            date=date.strftime('%Y-%m-%d')
            if emotion=="POSITIVO" and date not in self.positive_tweets:
                    self.positive_tweets[date]=[text]
                    #print(self.none_tweets)
            elif emotion=="POSITIVO" and date in self.positive_tweets:
                    self.positive_tweets[date].append(text)
                    #print(self.none_tweets)
    def set_negative_tweets(self):
        for text,date,emotion in self.tweets:
            date=date.strftime('%Y-%m-%d')
            if emotion=="NEGATIVO" and date not in self.negative_tweets:
                    self.negative_tweets[date]=[text]
                    #print(self.none_tweets)
            elif emotion=="NEGATIVO" and date in self.negative_tweets:
                    self.negative_tweets[date].append(text)
                    #print(self.none_tweets)
    def set_neutral_tweets(self):
        for text,date,emotion in self.tweets:
            date=date.strftime('%Y-%m-%d')
            if emotion=="NEUTRO" and date not in self.neutral_tweets:
                    self.neutral_tweets[date]=[text]
                    #print(self.none_tweets)
            elif emotion=="NEUTRO" and date in self.neutral_tweets:
                    self.neutral_tweets[date].append(text)
                    #print(self.none_tweets)


#cl=classifier(person_name="MIGUEL TORRUCO GARZA",tweets=get_tweets("MIGUEL TORRUCO GARZA",FROM_DATE="2023-12-01",TO_DATE="2024-01-07"))
cl=classifier(person_name="MIGUEL TORRUCO GARZA",tweets=get_tweets("MIGUEL TORRUCO GARZA"))
cl.set_none_tweets()
cl.set_positive_tweets()
cl.set_negative_tweets()
cl.set_neutral_tweets()
print(cl.positive_tweets)
print(cl.negative_tweets)
print(cl.neutral_tweets)
print(cl.none_tweets)







[('Viva Mexico Carbones !! ¡Echa un vistazo al vídeo de Miguel Torruco Garza! #TikTok https://t.co/9YUZh08JSc', datetime.date(2023, 11, 4), 'POSITIVO'), ('¡Echa un vistazo al vídeo de Miguel Torruco Garza! #TikTok https://t.co/BtQId4g6rm', datetime.date(2023, 11, 4), 'POSITIVO'), ('¡Echa un vistazo al vídeo de Miguel Torruco Garza! #TikTok https://t.co/RLESMpiq60', datetime.date(2023, 11, 4), 'POSITIVO'), ("Check out Miguel Torruco Garza's video! #TikTok https://t.co/FIKMs5IXAd. pinches perros de mierdaa", datetime.date(2023, 11, 4), 'NEGATIVO'), ("Check out Miguel Torruco Garza's video! #TikTok https://t.co/87FQCbnLdl", datetime.date(2023, 11, 4), 'NEGATIVO'), ("Check out Miguel Torruco Garza's video! #TikTok https://t.co/i4CdWBltzp", datetime.date(2023, 11, 4), 'POSITIVO'), ('¡Echa un vistazo al vídeo de Miguel Torruco Garza! #TikTok https://t.co/e0i9lU95Ty', datetime.date(2023, 11, 4), 'NEUTRO'), ('Aver perros derechangos ¡Echa un vistazo al vídeo de Miguel Torruco Garza! #TikTok ht

In [38]:
from db_functions import *
class words_analyzer:

    RELEVANT_TOPIC_TABLE="relevant_topic"

    def __init__(self,tweets_dic,emotion,person_name,date):
        self.person_name=person_name
        self.tweets_dic=tweets_dic
        self.date=date
        self.emotion=emotion
        self.tweets=None
    def set_date(self,date):
        self.date=date
    def get_meaningful_words(self):
        #self.set_date(date)
        list_of_tweets=self.tweets_dic[self.date]
        relevant_words=[]
        for tweet in list_of_tweets:
            #CLEAN TWEETS
            cleaned_tweet=clean_tweet(tweet)
            #print(cleaned_tweet)

            #TOKENIZE
            tokenized_text=tokenize_text(cleaned_tweet)
            #print(tokenized_text)

            #GET RELEVANT WORDS
            relevant_words.extend(tokenized_text)
        return relevant_words #RETURNS A LIST OF MEANINGFUL WORDS
    def words_frequency(self):
        words_list=self.get_meaningful_words()
        words_count={} 
        for word in words_list:
            if word in words_count:
                words_count[word]=words_count[word]+1
            else:
                words_count[word]=1
        return words_count #RETURNS A DICTIONARY INDICATING A SPECIFIC WORD ALONG WITH ITS FREQUENCY
    
    def most_frequent(self, top): 
        aux={}
        words_count=self.words_frequency()
        for element in words_count:
            aux[element]=words_count[element]#este se le irán eliminando elementos dinamicamente
            #recuerda que al ir recorriendo un diccionario se irá recorriendo por medio de la clave y no del valor
        
        aux2={}   #aqui se iran guardando los elementos de mayor frecuencia
        try:
            for element in range(top):
                clave_max=max(aux,key=aux.get) #GET THE MOST FREQUENT WORD IN THE AUX DICT
                aux2[clave_max]=aux[clave_max]
                aux.pop(clave_max)              
            return aux2                     #RETURNS A DICT WITH THE TOP(top) WORDS
        except:
            for element in range(len(aux)):
                clave_max=max(aux,key=aux.get)
                aux2[clave_max]=aux[clave_max]
                aux.pop(clave_max)
            return aux2                     #RETURNS A DICT WITH THE TOP(LEN(AUX)) WORDS

    def upload_top_relevant_words_to_db(self,dict):
        conn=connect_to_db()

        with conn: 
            with conn.cursor() as cur:

                query='''
                CREATE TABLE IF NOT EXISTS relevant_topic(
                collection_date DATE,
                cand_id int REFERENCES candidate(cand_id) ON DELETE CASCADE ON UPDATE cascade,
                word  TEXT ,
                frequency INT ,
                em_rate TEXT 
                CONSTRAINT check_not_null CHECK(collection_date is not null and word 
                is not null and frequency is not null  and em_rate is not null)
                )
                '''
                cur.execute(query)
        
        with conn:
            with conn.cursor() as cur:
                data=[]
                for word,count in dict.items():
                     data.append((self.date,get_candidate_id(self.person_name),word,count,self.emotion))
                query="INSERT INTO relevant_topic(collection_date,cand_id,word,frequency,em_rate) VALUES (%s,%s,%s,%s,%s)"
                cur.executemany(query,data)
        conn.close()

          
#analyzer=words_analyzer(tweets_dic=cl.neutral_tweets,emotion="NEUTRO",person_name=cl.person_name,date="2023-12-24")              
#print(analyzer.most_frequent(10))
#analyzer.upload_top_relevant_words_to_db(analyzer.most_frequent(10))


In [39]:
cl.set_dates_dict()
for emotion in list(cl.dates_dict.keys()):
    for date in cl.dates_dict[emotion]:
        if emotion=="POSITIVO":
            analyzer=words_analyzer(tweets_dic=cl.positive_tweets,emotion=emotion,person_name=cl.person_name,date=date)
            analyzer.upload_top_relevant_words_to_db(analyzer.most_frequent(10))
        elif emotion=="NEGATIVO":
            analyzer=words_analyzer(tweets_dic=cl.negative_tweets,emotion=emotion,person_name=cl.person_name,date=date)
            analyzer.upload_top_relevant_words_to_db(analyzer.most_frequent(10))
        elif emotion=="NEUTRO":
            analyzer=words_analyzer(tweets_dic=cl.neutral_tweets,emotion=emotion,person_name=cl.person_name,date=date)
            analyzer.upload_top_relevant_words_to_db(analyzer.most_frequent(10))
        elif emotion=="NULO":
            analyzer=words_analyzer(tweets_dic=cl.none_tweets,emotion=emotion,person_name=cl.person_name,date=date)
            analyzer.upload_top_relevant_words_to_db(analyzer.most_frequent(10))


In [40]:
print(cl.dates_dict)

{'POSITIVO': ['2023-11-04', '2023-11-05', '2023-11-09', '2023-11-13', '2023-11-14', '2023-11-11', '2023-11-12', '2023-11-02', '2023-11-03', '2023-11-10', '2023-12-24', '2023-12-29'], 'NEGATIVO': ['2023-11-04', '2023-11-05', '2023-11-13', '2023-11-01', '2023-11-02', '2023-11-03', '2023-11-09', '2023-12-24', '2023-12-27'], 'NEUTRO': ['2023-11-04', '2023-11-05', '2023-11-06', '2023-11-07', '2023-11-01', '2023-11-08', '2023-11-09', '2023-11-14', '2023-11-15', '2023-11-11', '2023-11-12', '2023-11-13', '2023-11-02', '2023-11-03', '2023-11-10', '2023-12-25', '2023-12-24', '2023-12-28', '2023-12-23', '2023-12-31', '2024-01-01'], 'NULO': ['2024-01-04', '2024-01-03', '2024-01-02', '2024-01-05', '2024-01-06', '2024-01-07']}
